In [1]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import time
from getpass import getpass
from ftplib import FTP_TLS
from ftplib import FTP
import ftplib

In [2]:
import pandas as pd
import re
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'

def extract_id(fileName):
    ogd_id = fileName.partition('_')[0]
    return ogd_id.replace('ogd', '')

def getInformation(url):

  csvList = []
  url = url.replace(" ","%20")
  req = Request(url)
  a = urlopen(req).read()
  soup = BeautifulSoup(a, 'html.parser')
  x = (soup.find_all('a'))

  for elem in x:
    stringElem = str(elem)

    if 'csv' in stringElem:
      fileName = re.search('href="(.*)"', stringElem).group(1)
      csvList.append([fileName, extract_id(fileName)])

  df = pd.DataFrame(csvList, columns= ['fileName', 'ogd_id'])
  print(df)
  return df
  #df.to_csv('files/info.csv')

df = getInformation(url)

                                            fileName ogd_id
0  ogd35_schweizerische_elektrizitaetsbilanz_mona...     35
1                                     ogd39_epec.csv     39
2                     ogd47_vollzugsresultate_pw.csv     47
3                ogd47_vollzugsresultate_pw_2020.csv     47
4                    ogd61_vollzugsresultate_lnf.csv     61
5               ogd61_vollzugsresultate_lnf_2020.csv     61
6           ogd90_potenzialstudien_waermenutzung.csv     90


In [4]:
# import urllib library
from urllib.request import urlopen

# import json
import json

url_dataPackage = 'https://www.uvek-gis.admin.ch/BFE/ogd/39/datapackage.json'

# store the response of URL
response = urlopen(url_dataPackage)

# storing the JSON response
# from url in data
data_json = json.loads(response.read())

# print the json response
print(data_json)

{'profile': 'tabular-data-package', 'resources': [{'name': 'ogd39_epec.csv', 'languages': ['de', 'fr'], 'path': 'https://www.uvek-gis.admin.ch/BFE/ogd/39/ogd39_epec.csv', 'profile': 'tabular-data-resource', 'schema': {'fields': [{'name': 'Kantonsname', 'type': 'string', 'format': 'default', 'title@de': 'Kantonsname', 'title@fr': '', 'description@de': 'Kantonsname', 'description@fr': ''}, {'name': 'Kantonskuerzel', 'type': 'string', 'format': 'default', 'title@de': 'Kantonskürzel', 'title@fr': '', 'description@de': 'Offizielles Kantonskürzel', 'description@fr': ''}, {'name': 'jahr', 'type': 'year', 'format': 'default', 'title@de': 'Jahr', 'title@fr': '', 'description@de': 'Jahr', 'description@fr': '', 'rdfType': 'http://www.w3.org/2006/time#Year'}, {'name': 'fr_0101', 'type': 'string', 'format': 'default', 'title@de': '1.1 - Kantonale Rechtsgrundlagen', 'title@fr': '', 'description@de': '1.1 Kantonale Energie- und Klimapolitik: Überblick der Rechtsgrundlagen - Kantonale Rechtsgrundlagen

In [ ]:
id = 35
fileName = 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv'
data_json['resources'][0]['path'] = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/' + fileName

In [ ]:
type(data_json)

dict

In [ ]:
json_data = json.dumps(data_json)

# print the Converted JSON object
print(json_data)

{"profile": "tabular-data-package", "resources": [{"name": "ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv", "path": "https://www.uvek-gis.admin.ch/BFE/ogd/staging/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv", "profile": "tabular-data-resource", "schema": {"fields": [{"name": "Jahr", "type": "year", "format": "default", "title@de": "Jahr", "title@fr": "Ann\u00e9e", "description@de": "Jahr der Aufnahme", "description@fr": "Ann\u00e9e d'acquisition"}, {"name": "Monat", "type": "integer", "format": "default", "title@de": "Monat", "title@fr": "Mois", "description@de": "Monat der Aufnahme", "description@fr": "Mois d'acquisition", "constraints": {"minimum": "1", "maximum": "12"}}, {"name": "Definitiv", "type": "boolean", "format": "default", "title@de": "", "title@fr": "", "description@de": "", "description@fr": "", "trueValues": ["1"], "falseValues": ["0"]}, {"name": "Erzeugung_laufwerk_GWh", "type": "integer", "format": "default", "title@de": "Erzeugung Laufwerk GW

In [ ]:
import json
with open('datapackageStaging.json', 'w', encoding='utf-8') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=4)

-----------------------------------------------------------------------------------------------

Load the datapackage content from the ogd directory, change the path, save new datapackage as _datapackageStaging.json_.

In [ ]:
df

,fileName,ogd_id
0,ogd35_schweizerische_elektrizitaetsbilanz_mona...,35
1,ogd39_epec.csv,39
2,ogd47_vollzugsresultate_pw.csv,47
3,ogd47_vollzugsresultate_pw_2020.csv,47
4,ogd61_vollzugsresultate_lnf.csv,61
5,ogd61_vollzugsresultate_lnf_2020.csv,61
6,ogd90_potenzialstudien_waermenutzung.csv,90


---------------------------------------------

In [ ]:
id = 35
fileName = 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv'
data_json['resources'][0]['path'] = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/' + fileName

json_data = json.dumps(data_json)

# print the Converted JSON object
print(json_data)

import json
with open('datapackageStaging.json', 'w', encoding='utf-8') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=4)

{"profile": "tabular-data-package", "resources": [{"name": "ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv", "path": "https://www.uvek-gis.admin.ch/BFE/ogd/staging/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv", "profile": "tabular-data-resource", "schema": {"fields": [{"name": "Jahr", "type": "year", "format": "default", "title@de": "Jahr", "title@fr": "Ann\u00e9e", "description@de": "Jahr der Aufnahme", "description@fr": "Ann\u00e9e d'acquisition"}, {"name": "Monat", "type": "integer", "format": "default", "title@de": "Monat", "title@fr": "Mois", "description@de": "Monat der Aufnahme", "description@fr": "Mois d'acquisition", "constraints": {"minimum": "1", "maximum": "12"}}, {"name": "Definitiv", "type": "boolean", "format": "default", "title@de": "", "title@fr": "", "description@de": "", "description@fr": "", "trueValues": ["1"], "falseValues": ["0"]}, {"name": "Erzeugung_laufwerk_GWh", "type": "integer", "format": "default", "title@de": "Erzeugung Laufwerk GW

---------------------------------------------

In [ ]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

    folderPath = url_ogd + row['ogd_id'] + '/'
    datapackagePath = folderPath + dataPackage

    #print(datapackagePath)

    # store the response of URL
    response = urlopen(datapackagePath)

    # storing the JSON response
    # from url in data
    data_json = json.loads(response.read())
    data_json['resources'][0]['path'] = url_ogd + staging + row['fileName']

    # print the json response
    print(data_json)

    with open(folderPath + 'datapackageStaging.json', 'w', encoding='utf-8') as f:
      json.dump(data_json, f, ensure_ascii=False, indent=4)

{'profile': 'tabular-data-package', 'resources': [{'name': 'ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'path': 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/ogd35_schweizerische_elektrizitaetsbilanz_monatswerte.csv', 'profile': 'tabular-data-resource', 'schema': {'fields': [{'name': 'Jahr', 'type': 'year', 'format': 'default', 'title@de': 'Jahr', 'title@fr': 'Année', 'description@de': 'Jahr der Aufnahme', 'description@fr': "Année d'acquisition"}, {'name': 'Monat', 'type': 'integer', 'format': 'default', 'title@de': 'Monat', 'title@fr': 'Mois', 'description@de': 'Monat der Aufnahme', 'description@fr': "Mois d'acquisition", 'constraints': {'minimum': '1', 'maximum': '12'}}, {'name': 'Definitiv', 'type': 'boolean', 'format': 'default', 'title@de': '', 'title@fr': '', 'description@de': '', 'description@fr': '', 'trueValues': ['1'], 'falseValues': ['0']}, {'name': 'Erzeugung_laufwerk_GWh', 'type': 'integer', 'format': 'default', 'title@de': 'Erzeugung Laufwerk GWh', 'title

FileNotFoundError: ignored

In [ ]:
import requests
r = requests.post('https://www.uvek-gis.admin.ch/BFE/ogd/staging/', json={"key": "value"})
r.status_code

404

--------------------------------------------------------------------------------

Connect to  FTP server to upload file -->  

In [ ]:
import ftplib
server = ftplib.FTP()
server.connect('10.73.220.213', 21)
server.login(getpass('Enter user: '), getpass('Enter password: '))

TimeoutError: ignored

In [ ]:
import os
import pandas as pd
import requests
import datetime
import zipfile
from ftplib import FTP_TLS
from ftplib import FTP
import ftplib

# passive mode should be on by default
#ftps = FTP("10.73.220.213").set_pasv(False)

ftps = ftplib.FTP_TLS()
ftps.connect("10.73.220.213", 21)

ftps.login(getpass('Enter user: '), getpass('Enter password: '))
file = open("datapackageStaging.json","rb")
ftps.storbinary("STOR https://www.uvek-gis.admin.ch/BFE/ogd/staging/datapackageStaging.json", file)
file.close()
ftps.close()


TimeoutError: ignored

---------------------------------------------

In [ ]:
import ftplib
import ssl


class ImplicitFTP_TLS(ftplib.FTP_TLS):
    """FTP_TLS subclass that automatically wraps sockets in SSL to support implicit FTPS."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._sock = None

    @property
    def sock(self):
        """Return the socket."""
        return self._sock

    @sock.setter
    def sock(self, value):
        """When modifying the socket, ensure that it is ssl wrapped."""
        if value is not None and not isinstance(value, ssl.SSLSocket):
            value = self.context.wrap_socket(value)
        self._sock = value

In [ ]:
#from ftp_tls import ImplicitFTP_TLS

ftps = ImplicitFTP_TLS()
ftps.connect("10.73.220.213", timeout=50)
ftps.login(getpass('Enter user: '), getpass('Enter password: '))

print('Connected to FTP...')

ftps.close()

TimeoutError: ignored

-----------------------------------------------------------------------

In [ ]:
class FTP_TLS_IgnoreHost(ftplib.FTP_TLS):
    def makepasv(self):
        _, port = super().makepasv()
        return self.host, port

ftp = FTP_TLS_IgnoreHost("10.73.220.213", getpass('Enter user: '), getpass('Enter password: '))

Enter user: ··········
Enter password: ··········


KeyboardInterrupt: ignored